In [ ]:
# python version: 3.8.5
import pandas as pd
import spacy
from scispacy.abbreviation import AbbreviationDetector
import re

In [ ]:
# Dataset
df = pd.read_json("inf_processed_all.json")

In [ ]:
nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe("abbreviation_detector")

def get_abbreviation_map(text):
    text=str(text)
    doc = nlp(text)
    abbr_dict = {}
    for abrv in doc._.abbreviations:
        abbr_dict[str(abrv)] = str(abrv._.long_form)
    return abbr_dict

def process_q_row(qa_list):
    qa_list=str(qa_list)
    abbr_map = get_abbreviation_map(qa_list)
    print(abbr_map)

    if abbr_map:
        # for i in range(len(qa_list)):
            for abbr, full in abbr_map.items():
                pattern = r'\b' + re.escape(abbr) + r'\b'
                qa_list = re.sub(pattern, full, qa_list) 
    return qa_list


In [ ]:
# Save abbreviation dictionary to dataframe
df['AbbrDict'] = df['Q&A'].apply(get_abbreviation_map)
# Save processed Q&A to dataframe
df["new_Q&A"] = df["Q&A"].apply(process_q_row)

In [ ]:
# save data to csv&json file 
df.to_csv('abbr_complement.csv', index=False)
df.to_json('abbr_complement.json', orient='records')